# 📚 1. Standard Library and Configuration

In [ ]:
# --- 1. SYSTEM & ENVIRONMENT CONFIGURATION ---
import os
import random
from warnings import filterwarnings

# Matikan log TensorFlow yang tidak perlu (Set sebelum import TF)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# Abaikan warnings
filterwarnings("ignore")


# --- Core Library ---
import numpy as np
import pandas as pd


# --- Visualization ---
import seaborn as sns
import matplotlib.pyplot as plt


# --- Scikit Learn ---
# Model Selection & Evaluation
from sklearn.model_selection import (
    KFold, 
    cross_validate, 
    cross_val_score, 
    train_test_split
)
from sklearn.metrics import (
    r2_score, 
    mean_squared_error, 
    mean_absolute_error
)

# Preprocessing & Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder, 
    StandardScaler, 
    RobustScaler
)


# --- Deep Learning Frameworks ---
# TensorFlow & Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense, 
    Dropout, 
    BatchNormalization,
    Activation
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, 
    ReduceLROnPlateau
)

# ML Models
import optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# --- Global Constant ---
N_SPLITS = 5
EPOCHS = 300
BATCH_SIZE = 8
TRAIN_PATH = '/kaggle/input/insurance/insurance.csv'

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    try:
        tf.config.experimental.enable_op_determinism()
    except AttributeError:
        print("Warning: tf.config.experimental.enable_op_determinism() tidak tersedia di versi TF ini.")

SEED = 42
set_seed(SEED)

print('Library and Configuration Ready!')

# 🛠️ 2. Utility Functions & Modular Pipeline

Bagian ini berisi kumpulan fungsi modular yang dirancang untuk menyederhanakan proses *end-to-end* machine learning, mulai dari visualisasi data hingga *Deep Learning*.

### 1. Feature Visualization (`plot_features`, `plot_numerical_correlation`)
Fungsi-fungsi ini digunakan untuk **Exploratory Data Analysis (EDA)**.
* `plot_features`: Secara otomatis mendeteksi tipe data dan membuat Histogram (untuk numerik) serta Bar Plot (untuk kategorikal) dalam satu *grid layout* yang rapi.
* `plot_numerical_correlation`: Menghasilkan *heatmap* korelasi Pearson untuk melihat hubungan antar fitur numerik.

### 2. Preprocessing (`get_preprocessor`)
Membangun `ColumnTransformer` standar menggunakan Scikit-Learn:
* **Numerik**: Di-*scale* menggunakan `StandardScaler`.
* **Kategorikal**: Di-*encode* menggunakan `OneHotEncoder`.

### 3. Hyperparameter Tuning (`tune_model`, Search Spaces)
Mesin optimasi menggunakan **Optuna** untuk mencari parameter terbaik bagi model Tree-Based (XGBoost & LightGBM).
* `get_xgb_search_space` & `get_lgbm_search_space`: Mendefinisikan rentang parameter yang akan diuji.
* `tune_model`: Menjalankan *trial* Optuna dengan validasi silang (Cross-Validation) untuk meminimalkan MAE.

### 4. Evaluation & Interpretation (`evaluate_final_model`, `plot_feature_importance`)
* `evaluate_final_model`: Melakukan evaluasi akhir yang robust menggunakan **5-Fold Cross-Validation** dan menampilkan metrik R2, MAE, dan RMSE.
* `plot_feature_importance`: Mengekstrak dan memvisualisasikan fitur mana yang paling berpengaruh terhadap prediksi model.

### 5. Deep Neural Network Engine (`build_dnn_model`, `run_dnn_cv`)
Implementasi khusus untuk Deep Learning menggunakan **Keras/TensorFlow**:
* **Arsitektur**: Menggunakan layer `Dense`, `BatchNormalization`, dan `Dropout` untuk mencegah overfitting.
* **Cross-Validation Manual**: `run_dnn_cv` menangani *looping* K-Fold secara manual agar preprocessing data (konversi ke numpy array) tidak bocor (*leakage*) antar fold.
* **Permutation Importance**: `explain_dnn_feature_importance` menghitung *feature importance* untuk DNN dengan cara mengacak kolom fitur satu per satu dan melihat dampaknya terhadap *error*.

In [ ]:
# --- To see the distribution for target features ---
def plot_target(series: pd.Series, log: bool = False) -> None:
    """
    Visualizes the distribution of the target variable using a Histogram and KDE.
    Optionally applies a log transformation to handle skewness.

    Args:
        series (pd.Series): The target variable data series (e.g., 'charges').
        log (bool): If True, applies natural logarithm transformation (np.log) 
                    to the series before plotting. Defaults to False.
    """
    # Apply log transformation if requested (useful for right-skewed data)
    if log:
        series = np.log(series)
        
    plt.figure(figsize=(15, 6))
    
    # Plot Histogram with Kernel Density Estimate (KDE)
    sns.histplot(series, kde=True, bins=50, edgecolor='black')
    
    # Customize plot aesthetics
    plt.grid(True, alpha=0.7)
    
    # Update title based on transformation status
    title_suffix = " (Log Transformed)" if log else ""
    plt.title(f'Target Distribution{title_suffix}', fontsize=20)
    
    plt.xlabel('Charges', fontsize=15)
    plt.ylabel('Counts', fontsize=15)
    
    plt.tight_layout()
    plt.show()

# --- Feature Visualization Utility ---

def plot_features(X: pd.DataFrame, numerical_features: list, categorical_features: list) -> None:
    """
    Generates histograms for numerical features and bar plots for categorical features
    to visualize data distribution.

    Args:
        X (pd.DataFrame): Input dataframe containing the data.
        numerical_features (list): List of column names corresponding to numerical data.
        categorical_features (list): List of column names corresponding to categorical data.
    """
    n_num = len(numerical_features)
    n_cat = len(categorical_features)
    
    # Determine grid dimensions based on the number of features
    max_cols = max(n_num, n_cat, 1)
    
    # Dynamic figure height: Allocate 4 units for each row (numerical/categorical)
    fig_height = 4 * ((1 if n_num > 0 else 0) + (1 if n_cat > 0 else 0))
    
    fig = plt.figure(figsize=(max_cols * 5, fig_height))
    gs = fig.add_gridspec(2, max_cols, hspace=0.4, wspace=0.3)
    
    # Plot Numerical Features
    if n_num > 0:
        print(f"Plotting {n_num} numerical histograms...")
        for i, col in enumerate(numerical_features):
            ax = fig.add_subplot(gs[0, i])
            
            # Limit bins to 50 or the number of unique values to prevent over-plotting
            bins = min(X[col].nunique(), 50) 
            # Enable Kernel Density Estimate (KDE) only if there are enough unique values
            kde = True if X[col].nunique() > 10 else False
            
            sns.histplot(X[col].dropna(), bins=bins, kde=kde, ax=ax, edgecolor='black')
            
            ax.set_title(f'Distribution: {col}', fontsize=12, fontweight='bold')
            sns.despine(ax=ax) # Remove top and right spines for a cleaner look

    # Plot Categorical Features
    if n_cat > 0:
        print(f"Plotting {n_cat} categorical bar plots...")
        # If numerical features exist, plot categorical on the second row (index 1)
        row_idx = 1 if n_num > 0 else 0 
        
        for i, col in enumerate(categorical_features):
            ax = fig.add_subplot(gs[row_idx, i])
            
            display_counts = X[col].value_counts()
            
            sns.barplot(x=display_counts.index, y=display_counts.values, ax=ax, 
                        palette='viridis')
            
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
            ax.set_title(f'Count: {col}', fontsize=12, fontweight='bold')
            sns.despine(ax=ax)

    plt.suptitle('Feature Distribution Visualization', fontsize=16, fontweight='heavy', y=0.98)
    # Adjust subplots to fit into the figure area nicely
    plt.tight_layout(rect=[0, 0, 1, 0.96]) 
    plt.show()


def plot_numerical_correlation(X: pd.DataFrame, numerical_features: list) -> None:
    """
    Computes and visualizes the Pearson correlation matrix for numerical features
    using a heatmap.

    Args:
        X (pd.DataFrame): Input dataframe.
        numerical_features (list): List of numerical column names to correlate.
    """
    # Calculate the pairwise correlation of columns, excluding NA/null values
    correlation_matrix = X[numerical_features].corr(method='pearson')
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        correlation_matrix, 
        annot=True,           # Write the data value in each cell
        fmt=".2f",            # String formatting code to use when adding annotations
        cmap='coolwarm',      # Diverging colormap (Red for pos, Blue for neg)
        vmin=-1, vmax=1,      # Anchor the colormap range
        center=0,             # Center the colormap at 0
        linewidths=.5,        
        cbar_kws={'label': 'Correlation Coefficient'}
    )
    
    plt.title('Numerical Features Correlation Heatmap (Pearson)', 
              fontsize=14, fontweight='bold')
    
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# --- Preprocessing Utility ---

def get_preprocessor(num_cols: list, cat_cols: list) -> ColumnTransformer:
    """
    Creates a sklearn ColumnTransformer to handle mixed types of data.
    
    Args:
        num_cols (list): List of numerical column names.
        cat_cols (list): List of categorical column names.
        
    Returns:
        ColumnTransformer: A transformer object ready to be used in a pipeline.
    """
    return ColumnTransformer(
        transformers=[
            # Apply OneHotEncoder to categorical cols; drop first to avoid dummy trap
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'), cat_cols),
            # Apply StandardScaler to numerical cols for normalization
            ('num', StandardScaler(), num_cols)
        ], remainder='drop' # Drop any columns not specified in transformers
    )

# --- Search Space Definitions ---

def get_xgb_search_space(trial: optuna.Trial) -> dict:
    """
    Defines the hyperparameter search space for XGBoost optimization.
    
    Args:
        trial (optuna.Trial): The Optuna trial object used to suggest hyperparameters.
        
    Returns:
        dict: A dictionary of hyperparameters for the XGBoost model.
    """
    return {
        'n_estimators': trial.suggest_int('n_estimators', 200, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'n_jobs': -1, 'random_state': 42
    }

def get_lgbm_search_space(trial: optuna.Trial) -> dict:
    """
    Defines the hyperparameter search space for LightGBM optimization.
    
    Args:
        trial (optuna.Trial): The Optuna trial object used to suggest hyperparameters.
        
    Returns:
        dict: A dictionary of hyperparameters for the LightGBM model.
    """
    return {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'verbosity': -1, 'n_jobs': 1, 'random_state': 42
    }

# --- Tuning Engine ---

def tune_model(model_cls, search_space_func, X, y, preprocessor, n_trials: int=50, study_name: str="optimization") -> dict:
    """
    Generic function to run Optuna hyperparameter tuning using Cross-Validation.

    Args:
        model_cls: The model class constructor (e.g., XGBRegressor).
        search_space_func: Function that returns the parameter dictionary for a trial.
        X, y: Training data features and target.
        preprocessor: The sklearn preprocessor object/transformer.
        n_trials (int): Number of optimization trials to run.
        study_name (str): Name/Tag for the optuna study logging.
        
    Returns:
        dict: The best hyperparameters found, combined with static parameters.
    """
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    print(f"🚀 Starting Tuning for {study_name}...")

    def objective(trial):
        # Retrieve dynamic parameters from the search space function
        params = search_space_func(trial)
        # Instantiate the model with current trial parameters
        model = model_cls(**params)        
        
        # Create a temporary pipeline for validation
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        # perform 3-fold CV to speed up the tuning process
        scores = cross_val_score(pipeline, X, y, cv=3, 
                                 scoring='neg_mean_absolute_error', n_jobs=-1)
        
        # Optuna minimizes the objective, so we return negative MAE (to minimize error)
        # Note: cross_val_score returns negative values for errors, so -scores.mean() is positive error.
        # Actually, if direction is 'minimize', we should return the Error (positive).
        # cross_val_score gives negative error (e.g. -5). -(-5) = 5. We want to minimize 5.
        return -scores.mean()

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    print(f"✅ {study_name} Best MAE: {study.best_value:.4f}")
    
    # Combine best params with static params (random_state, n_jobs) for final use
    final_params = study.best_params.copy()
    final_params.update({'random_state': 42, 'n_jobs': -1})
    if study_name == "LightGBM": final_params['verbosity'] = -1
        
    return final_params

# --- Evaluation Engine ---

def evaluate_final_model(model_cls, best_params: dict, X, y, preprocessor) -> Pipeline:
    """
    Creates the final model pipeline with best parameters and runs robust 
    5-Fold Cross-Validation evaluation.

    Args:
        model_cls: The model class constructor.
        best_params (dict): Dictionary of optimized hyperparameters.
        X, y: Training data.
        preprocessor: The sklearn preprocessor object.
        
    Returns:
        Pipeline: The full, untrained pipeline object (ready for final fitting).
    """
    
    print(f"\n📊 EVALUATING FINAL MODEL: {model_cls.__name__}")
    
    final_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model_cls(**best_params))
    ])
    
    # Perform 5-Fold CV with multiple scoring metrics
    results = cross_validate(
        final_pipeline, X, y, 
        cv=KFold(n_splits=5, shuffle=True, random_state=42),
        scoring={'r2': 'r2',
                 'mae': 'neg_mean_absolute_error',
                 'rmse': 'neg_root_mean_squared_error'},
        n_jobs=1
    )
    
    # Output scores for each fold
    print("-" * 60)
    for i in range(len(results['test_r2'])):
        r2 = results['test_r2'][i]
        mae = -results['test_mae'][i]
        rmse = -results['test_rmse'][i]
        print(f"  Fold {i+1} -> R2: {r2:.4f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")

    # Output the mean scores
    print("-" * 60)
    print(f"  AVG R2   : {results['test_r2'].mean():.4f}")
    print(f"  AVG MAE  : {-results['test_mae'].mean():.2f}")
    print(f"  AVG RMSE : {-results['test_rmse'].mean():.2f}")
    print("-" * 60)
    
    return final_pipeline

def plot_feature_importance(pipeline: Pipeline, X, y, model_name: str="Model") -> pd.DataFrame:
    """
    Fits the pipeline to the full dataset, extracts feature importance scores, 
    and generates a bar plot.

    Args:
        pipeline (Pipeline): The full model pipeline (preprocessor + regressor).
        X, y: Training data.
        model_name (str): Name of the model for visualization titles.
        
    Returns:
        pd.DataFrame: Dataframe containing feature names and their importance scores.
    """
    print(f"⚙️ Sedang memproses Feature Importance untuk: {model_name}...")
    
    # Fit Pipeline on the entire dataset to calculate final importances
    pipeline.fit(X, y)
    
    # Access individual steps from the pipeline
    model = pipeline.named_steps['regressor']
    preprocessor = pipeline.named_steps['preprocessor']
    
    # Retrieve transformed feature names (handling OneHotEncoding prefixes)
    raw_feature_names = preprocessor.get_feature_names_out()
    # Clean the 'cat__' or 'num__' prefix added by ColumnTransformer
    clean_feature_names = [name.split('__')[-1] for name in raw_feature_names]
    
    # Check if the model supports native feature importance
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        print(f"⚠️ Model {model_name} tidak mendukung feature_importances_.")
        return None

    # Create a DataFrame for easier plotting and sorting
    importance_df = pd.DataFrame({
        'feature': clean_feature_names,
        'importance': importances
    }).sort_values(by='importance', ascending=False)

    # Visualization
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=importance_df, # You can slice this to top N (e.g., importance_df.head(15))
        x='importance', 
        y='feature',
        palette='mako' # Elegant & deep palette for aesthetics
    )
    plt.title(f'Top Feature Importance - {model_name}', fontsize=14, fontweight='bold')
    plt.xlabel('Importance Score')
    plt.ylabel('Features')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    return importance_df


# --- DNN Architecture Definition ---

def build_dnn_model(input_dim: int, learning_rate: float = 0.01) -> Sequential:
    """
    Constructs and compiles a Keras Deep Neural Network architecture for regression.
    
    Args:
        input_dim (int): Number of input features (after preprocessing).
        learning_rate (float): Learning rate for the Adam optimizer.
        
    Returns:
        Sequential: A compiled Keras model.
    """
    model = Sequential([
        # Layer 1: Input layer with 128 neurons
        Dense(128, input_dim=input_dim),
        BatchNormalization(), # Normalize inputs to speed up training
        Activation('relu'),
        Dropout(0.3), # Randomly drop 30% of neurons to prevent overfitting

        # Layer 2: Hidden layer with 64 neurons
        Dense(64),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),

        # Layer 3: Hidden layer with 16 neurons
        Dense(16, activation='relu'),
        
        # Output Layer: Single neuron for regression output
        Dense(1, activation='linear')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

# --- DNN Cross-Validation Engine ---

def run_dnn_cv(X, y, num_cols, cat_cols, n_splits=5, epochs=100, batch_size=32):
    """
    Executes K-Fold Cross-Validation specifically for the DNN model.
    Note: Handles preprocessing inside the loop to prevent data leakage, as DNNs
    require numpy arrays (not pandas DFs) as input.
    
    Args:
        X, y: Input data features and target.
        num_cols, cat_cols: Lists of feature names for preprocessing.
        n_splits (int): Number of K-Fold splits.
        epochs (int): Max training epochs per fold.
        batch_size (int): Batch size for training.
        
    Returns:
        dict: A dictionary containing average scores (R2, RMSE, MAE).
    """
    print(f"🧠 Memulai {n_splits}-Fold Cross-Validation untuk DNN...")
    print(f"   (Training | Max Epochs: {epochs} | Batch Size: {batch_size})")
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Dictionary to store metrics from each fold
    scores = {'r2': [], 'rmse': [], 'mae': []}
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        # A. Split Data using indices
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # B. Preprocessing (Fresh instance per fold to avoid leakage)
        fold_preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'), cat_cols),
                ('num', StandardScaler(), num_cols)
            ], remainder='drop'
        )
        
        # Fit on train, transform on val
        X_train_proc = fold_preprocessor.fit_transform(X_train)
        X_val_proc = fold_preprocessor.transform(X_val)
        
        # C. Build Model with correct input dimension
        model = build_dnn_model(input_dim=X_train_proc.shape[1])
        
        # D. Define Callbacks for training stability
        callbacks = [
            # Stop training if validation loss stops improving
            EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=0),
            # Reduce learning rate when a metric has stopped improving
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=0)
        ]
        
        # E. Train the model (Silent mode)
        model.fit(
            X_train_proc, y_train,
            validation_data=(X_val_proc, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=0 
        )
        
        # F. Evaluate on Validation Set
        y_pred = model.predict(X_val_proc, verbose=0).flatten()
        
        r2 = r2_score(y_val, y_pred)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mae = mean_absolute_error(y_val, y_pred)
        
        scores['r2'].append(r2)
        scores['rmse'].append(rmse)
        scores['mae'].append(mae)
        
        print(f"   ✅ Fold {fold+1}: R2={r2:.4f} | RMSE={rmse:.2f} | MAE={mae:.2f}")
        
    # Calculate Averages across all folds
    final_results = {
        'R2 Score': np.mean(scores['r2']),
        'RMSE': np.mean(scores['rmse']),
        'MAE': np.mean(scores['mae'])
    }
    
    print("\n" + "="*50)
    print(f"RATA-RATA PERFORMA DNN ({n_splits} Folds)")
    print("="*50)
    print(f"R2   : {final_results['R2 Score']:.4f}")
    print(f"MAE  : {final_results['MAE']:.4f}")
    print(f"RMSE : {final_results['RMSE']:.4f}")
    print("="*50)
    
    return final_results

def train_final_dnn(X, y, num_cols, cat_cols, epochs=100, batch_size=32):
    """
    Trains a single DNN model on the entire dataset for feature importance analysis.
    
    Returns:
        tuple: (trained_keras_model, processed_features_numpy, feature_names_list)
    """
    print("⚙️ Melatih Final DNN pada seluruh dataset...")
    
    # 1. Preprocessing
    preprocessor = get_preprocessor(num_cols, cat_cols)
    X_proc = preprocessor.fit_transform(X)
    
    # 2. Extract Feature Names from the preprocessor
    raw_names = preprocessor.get_feature_names_out()
    feature_names = [name.split('__')[-1] for name in raw_names]
    
    # 3. Build & Fit Model
    model = build_dnn_model(input_dim=X_proc.shape[1])
    model.fit(X_proc, y, epochs=epochs, batch_size=batch_size, verbose=0)
    
    print("✅ Final DNN selesai dilatih.")
    return model, X_proc, feature_names

# --- Visualization: DNN Permutation Importance ---

def explain_dnn_feature_importance(model, X_processed, y_true, feature_names):
    """
    Computes and visualizes Feature Importance for Deep Learning models using
    Permutation Importance (shuffling features one by one to measure error increase).
    
    Args:
        model: Trained Keras model.
        X_processed: Preprocessed input data (numpy array).
        y_true: True target values.
        feature_names: List of feature names corresponding to X_processed columns.
        
    Returns:
        pd.DataFrame: Dataframe of feature importance scores.
    """
    print("🔍 Menghitung DNN Feature Importance (ini mungkin memakan waktu)...")
    
    # 1. Calculate Baseline MAE (Error with original data)
    y_pred_base = model.predict(X_processed, verbose=0).flatten()
    baseline_mae = mean_absolute_error(y_true, y_pred_base)
    
    importances = []
    
    # 2. Permutation Loop
    # Iterate over each feature column to calculate its importance
    for i in range(X_processed.shape[1]):
        X_permuted = X_processed.copy()
        np.random.shuffle(X_permuted[:, i]) # Randomly shuffle column i
        
        # Predict using the shuffled data
        y_pred_perm = model.predict(X_permuted, verbose=0).flatten()
        perm_mae = mean_absolute_error(y_true, y_pred_perm)
        
        # Importance = How much did the error increase compared to baseline?
        importances.append(perm_mae - baseline_mae)
        
    # 3. Create DataFrame
    df_imp = pd.DataFrame({
        'Feature': feature_names,
        'Importance_MAE': importances
    }).sort_values(by='Importance_MAE', ascending=False)
    
    # 4. Visualization
    plt.figure(figsize=(10, 8))
    sns.barplot(
        data=df_imp,
        x='Importance_MAE', 
        y='Feature',
        palette='magma'
    )
    plt.title(f'DNN Feature Importance\n(Baseline MAE: {baseline_mae:.2f})', fontsize=14, fontweight='bold')
    plt.xlabel('Increase in MAE (Error) when feature is shuffled')
    plt.ylabel('Features')
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return df_imp

# 📊 3. Load Data

## Dataset Penelitian: Medical Cost Personal Dataset

Dataset yang digunakan dalam penelitian merupakan **Medical Cost Personal Dataset** yang diperoleh dari laman Kaggle (sumber: [Kaggle](https://www.kaggle.com/datasets/mirichoi0218/insurance/data)). Dataset ini berisi data individu yang diasuransikan, dengan tujuan untuk menganalisis faktor-faktor yang memengaruhi besarnya klaim medis (*medical charges*) yang ditanggung oleh perusahaan asuransi kesehatan.

Secara keseluruhan, dataset ini terdiri atas **1.338 observasi** dan **7 variabel** yang mencakup karakteristik demografis, kondisi kesehatan, serta informasi biaya klaim. Adapun penjelasan tiap variabel adalah sebagai berikut.

---

### Deskripsi Variabel

* **age**: Usia tertanggung utama (*primary beneficiary*) yang diasuransikan. Umur merupakan faktor risiko utama dalam klaim kesehatan karena berkorelasi positif dengan frekuensi dan keparahan klaim.
* **sex**: Jenis kelamin tertanggung, terdiri atas kategori *female* dan *male*. Variabel ini digunakan untuk menilai adanya perbedaan biaya klaim antara laki-laki dan perempuan.
* **bmi**: *Body Mass Index* (BMI), yaitu indeks massa tubuh yang dihitung dari rasio berat badan terhadap tinggi badan (kg/m²). Nilai ideal berada pada kisaran 18,5–24,9. BMI digunakan sebagai indikator risiko kesehatan seperti obesitas yang berpotensi meningkatkan biaya klaim.
* **children**: Jumlah anak atau tanggungan yang tercakup dalam polis asuransi kesehatan. Variabel ini menunjukkan beban keluarga dalam perlindungan asuransi.
* **smoker**: Status kebiasaan merokok tertanggung (*yes* atau *no*). Faktor ini sangat berpengaruh terhadap besarnya premi maupun klaim karena berkaitan langsung dengan risiko penyakit kronis.
* **region**: Wilayah tempat tinggal tertanggung di Amerika Serikat, dengan kategori *northeast*, *southeast*, *southwest*, dan *northwest*. Variabel ini dapat mencerminkan perbedaan biaya layanan kesehatan antar wilayah.
* **charges**: Total biaya medis individual yang ditagihkan kepada perusahaan asuransi (*individual medical costs billed by health insurance*). Variabel ini menjadi variabel target (*dependent variable*) yang akan diprediksi dalam penelitian ini.

---

Dengan karakteristik tersebut, dataset ini dinilai sesuai untuk mendukung analisis faktor-faktor yang memengaruhi besarnya klaim asuransi kesehatan menggunakan pendekatan *machine learning*. Selain itu, variabel-variabelnya memungkinkan untuk dilakukan interpretasi dari sudut pandang aktuaria, terutama dalam konteks *risk classification* dan *expected claim cost estimation*.

In [ ]:
insurance = pd.read_csv(TRAIN_PATH)
insurance.head()

In [ ]:
print('Statistic descriptive of numerical features\n')
print(insurance.describe())

print('\nStatistic descriptive of categorical features\n')
print(insurance.describe(include='object'))

print('\nNumber of missing value each feature\n')
print(insurance.isna().sum())

In [ ]:
# Check duplicates → drop → verify
print("Missing value before:", insurance.duplicated().sum())

insurance.drop_duplicates(inplace=True)
print("Missing value after :", insurance.duplicated().sum())

# 🔎 4. Exploratory Data Analysis

In [ ]:
# Devide datasets into two part, predictor and target

X = insurance.drop(columns=['charges'])
y = insurance.charges

In [ ]:
plot_target(y, log=False)
plot_target(y, log=True)

Distribusi charges menunjukkan pola yang sangat right-skewed, di mana sebagian besar individu membayar biaya asuransi dalam rentang rendah hingga menengah (sekitar di bawah 15.000), sementara hanya sebagian kecil yang memiliki biaya sangat tinggi hingga lebih dari 60.000. Ekor panjang di sisi kanan menandakan adanya outliers atau kelompok kecil dengan biaya medis ekstrem, yang kemungkinan besar dipengaruhi oleh faktor seperti status perokok atau kondisi kesehatan tertentu. Pola ini juga menunjukkan bahwa variabel target tidak berdistribusi normal, sehingga transformasi seperti log-transform dapat membantu model regresi untuk mempelajari pola dengan lebih stabil dan mengurangi efek ekstrem dari nilai-nilai tinggi.

In [ ]:
numerical_features=X.select_dtypes(include=np.number).columns
categorical_features=X.select_dtypes(include='object').columns

plot_features(X, numerical_features, categorical_features)

Distribusi umur (age) tampak cukup merata dari usia 20 hingga 60 tahun tanpa pola khusus, menunjukkan tidak ada dominasi kelompok umur tertentu. Fitur BMI memiliki pola mendekati distribusi normal dengan pusat di sekitar 30, menunjukkan mayoritas peserta memiliki BMI overweight. Jumlah anak (children) didominasi oleh nilai 0–1, menandakan sebagian besar individu tidak memiliki anak atau hanya satu. Komposisi jenis kelamin (sex) relatif seimbang antara laki-laki dan perempuan. Status perokok (smoker) sangat tidak seimbang, dengan mayoritas besar adalah non-smoker. Distribusi wilayah (region) merata di empat region tanpa ada dominasi signifikan, sehingga tidak ada sampling bias besar dari wilayah.

In [ ]:
plot_numerical_correlation(X, numerical_features)

Heatmap menunjukkan bahwa hubungan antar fitur numerik (age, bmi, children) sangat lemah, dengan koefisien korelasi yang semuanya mendekati nol. Age hanya memiliki korelasi kecil dengan BMI, dan jumlah anak hampir tidak berhubungan dengan dua fitur lainnya. Ini mengindikasikan bahwa ketiga fitur tersebut cenderung berdiri sendiri dan tidak memiliki hubungan linear yang kuat satu sama lain, sehingga masing-masing kemungkinan berkontribusi secara independen terhadap variabel target (biaya insurance) dan tidak menimbulkan masalah multikolinearitas.

# 📈 5. Modeling

In [ ]:
high_cardinality_cols = [features for features in categorical_features if X[features].nunique()>10]
print('High cardinality column: ', high_cardinality_cols)

Semua variabel kategorikal tidak ada yang memiliki cardinality yang tinggi, semua variabel kategorik akan dilakukan One Hot Encoding

## Feature Engineering

Untuk membantu model menangkap pola risiko yang kompleks dan non-linier, kita membuat variabel-variabel turunan berikut:

* **`bmi_encoded` (Kategori BMI)**
    * **Tujuan:** Mengubah variabel numerik kontinu (`bmi`) menjadi variabel kategori ordinal (0, 1, 2, 3).
    * **Penjelasan:** Hubungan BMI dengan biaya kesehatan seringkali bertingkat (*step-function*), bukan garis lurus. Pengelompokan ini membantu model membedakan risiko antara kategori berat badan (misal: *Underweight, Normal, Overweight, Obese*) secara lebih tegas.
    * *Dibuat dari: `bmi`*

* **`smoker_obese_interaction` (Interaksi Perokok & Obesitas)**
    * **Tujuan:** Membuat fitur biner yang bernilai `1` jika seseorang adalah perokok **DAN** memiliki BMI tinggi (*Overweight/Obese* - kategori 2 atau 3).
    * **Penjelasan:** Ini adalah "Double Whammy Effect". Dalam aktuaria, risiko kesehatan perokok yang juga obesitas seringkali **multiplikatif**, bukan aditif. Biaya mereka biasanya jauh lebih tinggi dibandingkan yang hanya perokok atau hanya obesitas.
    * *Dibuat dari: `smoker` dan `bmi_encoded`*

* **`has_dependent_or_high_risk` (Tanggungan atau Risiko Gaya Hidup)**
    * **Tujuan:** Membuat fitur biner yang menandai jika seseorang punya anak **ATAU** merupakan perokok.
    * **Penjelasan:** Menggabungkan dua faktor pemicu biaya berbeda: beban finansial keluarga (anak) atau beban risiko kesehatan (rokok). Ini mengelompokkan profil "High Liability" secara umum.
    * *Dibuat dari: `children` dan `smoker`*

* **`smoker_age_interaction` (Interaksi Usia Khusus Perokok)**
    * **Tujuan:** Menyimpan nilai umur asli hanya jika orang tersebut perokok (jika tidak, nilainya 0).
    * **Penjelasan:** Dampak penuaan pada tubuh perokok mungkin lebih merusak (akselerasi biaya) dibandingkan non-perokok. Fitur ini mengizinkan model memberikan "bobot" atau koefisien yang berbeda untuk umur pada kelompok perokok.
    * *Dibuat dari: `smoker` dan `age`*

* **`smoker_obese_aged_interaction` (Risiko Ekstrem: Tua, Merokok, Gemuk)**
    * **Tujuan:** Fitur biner untuk segmen risiko sangat tinggi: Perokok + BMI Tinggi + Usia di atas 45 tahun.
    * **Penjelasan:** Ini mengisolasi kelompok "*Very High Risk*". Seringkali model gagal memprediksi nilai ekstrem (*outlier*) yang sangat mahal. Fitur ini memberi sinyal khusus pada model bahwa kelompok ini kemungkinan besar memiliki klaim raksasa.
    * *Dibuat dari: `smoker`, `bmi_encoded`, dan `age`*

* **`bmi_smoker_continuous` (Interaksi BMI Kontinu Khusus Perokok)**
    * **Tujuan:** Menyimpan nilai BMI asli hanya jika orang tersebut perokok.
    * **Penjelasan:** Serupa dengan interaksi usia, kenaikan 1 poin BMI pada perokok mungkin berdampak lebih besar terhadap biaya daripada kenaikan 1 poin BMI pada non-perokok (efek sinergis pada jantung/paru).
    * *Dibuat dari: `smoker` dan `bmi`*

* **`age_sq` (Usia Kuadrat)**
    * **Tujuan:** *Polynomial feature* untuk menangkap hubungan kuadratik pada usia.
    * **Penjelasan:** Biaya kesehatan biasanya tidak naik secara linier seiring bertambahnya usia, melainkan melengkung ke atas (eksponensial/kuadratik) terutama di usia tua.
    * *Dibuat dari: `age`*

* **`bmi_sq` (BMI Kuadrat)**
    * **Tujuan:** *Polynomial feature* untuk menangkap hubungan kuadratik pada BMI.
    * **Penjelasan:** Risiko kesehatan mungkin naik drastis (kurva berbentuk J atau U) pada angka BMI yang sangat ekstrem.
    * *Dibuat dari: `bmi`*

* **`age_bmi_interaction` (Interaksi Usia dan BMI)**
    * **Tujuan:** Perkalian antara usia dan BMI.
    * **Penjelasan:** Dampak obesitas mungkin menjadi lebih parah seiring bertambahnya usia. Menjadi gemuk di usia 50 tahun mungkin membawa risiko komplikasi (dan biaya) yang lebih besar daripada gemuk di usia 20 tahun.
    * *Dibuat dari: `age` dan `bmi`*

* **`sex_smoker_interaction` (Interaksi Gender & Rokok)**
    * **Tujuan:** Fitur biner khusus untuk Laki-laki yang Merokok.
    * **Penjelasan:** Menguji hipotesis apakah ada perbedaan pola klaim antara perokok pria dan wanita (misalnya karena perbedaan intensitas merokok atau kerentanan biologis).
    * *Dibuat dari: `sex` dan `smoker`*

* **`non_smoker_high_risk` (Risiko Tinggi Non-Perokok)**
    * **Tujuan:** Mengidentifikasi orang yang **bukan** perokok, tapi Tua (>45) dan Berat Badan Berlebih.
    * **Penjelasan:** Menangkap segmen risiko murni dari faktor metabolik dan penuaan tanpa pengaruh rokok. Ini membantu model membedakan sumber risiko biaya.
    * *Dibuat dari: `smoker`, `bmi_encoded`, dan `age`*

* **`log_charges` (Transformasi Log pada Target)**
    * **Tujuan:** Membuat variabel target baru dengan menerapkan transformasi logaritma natural (`np.log`) pada variabel `charges` asli.
    * **Penjelasan:** Distribusi data `charges` asli sangat miring ke kanan (*right-skewed*). Distribusi yang miring ini melanggar asumsi banyak model dan dapat menurunkan akurasi. `log_charges` membuat distribusi target lebih normal (simetris).
    * *Dibuat dari: `charges`*

In [ ]:
bins = [-np.inf, 18.5, 25, 30, np.inf]
labels = [0, 1, 2, 3]

insurance['bmi_encoded'] = pd.cut(insurance['bmi'], bins=bins, labels=labels)

insurance['smoker_obese_interaction'] = np.where(
    (insurance['smoker'] == 'yes') & (insurance['bmi_encoded'].isin([2, 3])), 
    1, 0
)

insurance['has_dependent_or_high_risk'] = np.where(
    (insurance['children'] > 0) | (insurance['smoker'] == 'yes'), 
    1, 0
)


insurance['smoker_age_interaction'] = np.where(
    insurance['smoker'] == 'yes',
    insurance['age'],
    0
)

insurance['smoker_obese_aged_interaction'] = np.where(
    (insurance['smoker'] == 'yes') & 
    (insurance['bmi_encoded'].isin([2, 3])) & 
    (insurance['age'] > 45),
    1, 0
)

insurance['bmi_smoker_continuous'] = np.where(
    insurance['smoker'] == 'yes',
    insurance['bmi'],
    0
)
insurance['age_sq'] = insurance['age'] ** 2

insurance['bmi_sq'] = insurance['bmi'] ** 2
insurance['age_bmi_interaction'] = insurance['age'] * insurance['bmi']
insurance['sex_smoker_interaction'] = np.where(
    (insurance['smoker'] == 'yes') & (insurance['sex'] == 'male'),
    1, 0
)
insurance['non_smoker_high_risk'] = np.where(
    (insurance['smoker'] == 'no') & 
    (insurance['bmi_encoded'].isin([2, 3])) & 
    (insurance['age'] > 45),
    1, 0
)

insurance['log_charges'] = np.log(insurance['charges'])

In [ ]:
insurance.info()

## Data Setup & Preprocessing Initialization

Tahap ini bertujuan untuk mempersiapkan data latih dan menginisialisasi komponen *preprocessing* menggunakan fungsi utilitas modular yang telah kita buat sebelumnya.

### 1. Feature Selection & Target Definition

Kita memisahkan kolom menjadi fitur (X) dan target (y) secara otomatis agar prosesnya dinamis:

* **`DROP_COLS`**: Mendefinisikan kolom target (`charges`) dan variannya (`log_charges`) untuk dikecualikan dari daftar fitur agar tidak terjadi kebocoran data (*data leakage*).
* **`NUM_FEATS` & `CAT_FEATS`**: Menggunakan `select_dtypes` untuk memisahkan nama kolom numerik dan kategorikal secara otomatis, dikurangi kolom yang ada di `DROP_COLS`.
* **`X` & `y`**:
    * `X`: Menggabungkan daftar fitur numerik dan kategorikal sebagai input model.
    * `y`: Menggunakan `charges` (nilai asli) sebagai target prediksi.

### 2. Preprocessor Initialization

* **`my_preprocessor`**: Kita memanggil fungsi utilitas **`get_preprocessor(NUM_FEATS, CAT_FEATS)`** yang telah kita definisikan di bagian *Utility Functions*.
* Fungsi ini secara otomatis membungkus logika transformasi standar:
    * **Numerik**: Diterapkan **`StandardScaler`** untuk menstandarisasi skala data.
    * **Kategorikal**: Diterapkan **`OneHotEncoder`** untuk mengubah data teks menjadi format numerik biner.

In [ ]:
# Setup Data
DROP_COLS = ['log_charges', 'charges'] 
NUM_FEATS = insurance.select_dtypes(include=np.number).columns.difference(DROP_COLS)
CAT_FEATS = insurance.select_dtypes(include='object').columns.difference(DROP_COLS)

X = insurance[list(NUM_FEATS) + list(CAT_FEATS)]
y = insurance['charges']

# preprocessor
my_preprocessor = get_preprocessor(NUM_FEATS, CAT_FEATS)

## Model Training & Optimization Phase

Pada tahap ini, kita menjalankan proses pelatihan model secara sistematis menggunakan *workflow* modular yang telah kita bangun. Kita akan membandingkan dua algoritma *Gradient Boosting* terpopuler: **XGBoost** dan **LightGBM**.

Setiap model akan melalui dua fase utama: **Tuning** dan **Evaluation**.

### 1. Phase A: Hyperparameter Tuning (`tune_model`)
Kita menggunakan fungsi utilitas `tune_model` yang memanfaatkan **Optuna** untuk mencari kombinasi parameter terbaik secara otomatis.

* **Strategi Pencarian:** Fungsi ini memanggil `get_xgb_search_space` atau `get_lgbm_search_space` untuk menentukan rentang parameter yang akan diuji (misalnya: `learning_rate`, `max_depth`, `num_leaves`).
* **Metode Validasi:** Menggunakan **3-Fold Cross-Validation** (seperti yang didefinisikan di dalam fungsi `tune_model`) untuk mempercepat proses pencarian tanpa mengorbankan validitas data.
* **Optimasi:** Optuna akan menjalankan **100 trials**, mencoba meminimalkan skor *Mean Absolute Error* (MAE).

### 2. Phase B: Robust Evaluation (`evaluate_final_model`)
Setelah parameter terbaik (`best_params`) ditemukan, kita tidak langsung mempercayai hasil tuning tersebut begitu saja. Kita memvalidasinya ulang menggunakan fungsi `evaluate_final_model`.

* **Pipeline Final:** Parameter terbaik disuntikkan ke dalam model baru dan dibungkus kembali dengan *preprocessor*.
* **Evaluasi Ketat:** Kita meningkatkan validasi menjadi **5-Fold Cross-Validation**. Ini memberikan estimasi performa yang lebih stabil dan tidak bias dibandingkan saat fase tuning.
* **Metrik Lengkap:** Fungsi ini akan mencetak skor rata-rata untuk **R2** (daya jelaskan model), **MAE** (kesalahan absolut), dan **RMSE** (kesalahan kuadratik) untuk memastikan model benar-benar robust sebelum digunakan.

In [ ]:
# === 1. XGBoost Workflow ===
# Step A: Tune
xgb_best_params = tune_model(
    model_cls=XGBRegressor, 
    search_space_func=get_xgb_search_space, 
    X=X, y=y, preprocessor=my_preprocessor, 
    n_trials=100, study_name="XGBoost"
)

# Step B: Evaluate
xgb_pipeline = evaluate_final_model(XGBRegressor, xgb_best_params, X, y, my_preprocessor)


# === 2. LightGBM Workflow ===
# Step A: Tune
lgbm_best_params = tune_model(
    model_cls=LGBMRegressor, 
    search_space_func=get_lgbm_search_space, 
    X=X, y=y, preprocessor=my_preprocessor, 
    n_trials=100, study_name="LightGBM"
)

# Step B: Evaluate
lgbm_pipeline = evaluate_final_model(LGBMRegressor, lgbm_best_params, X, y, my_preprocessor)

## Model Evaluation & Interpretation

Berdasarkan hasil *Hyperparameter Tuning* dan *5-Fold Cross-Validation*, berikut adalah analisis performa model prediksi biaya kesehatan (*charges*):

### 1. Perbandingan Model (XGBoost vs LightGBM)

| Metric | XGBoost (Winner) 🏆 | LightGBM | Selisih |
| :--- | :--- | :--- | :--- |
| **R2 Score** | **0.8584** | 0.8516 | XGBoost +0.68% |
| **MAE** | **`$2,454.01`** | `$2,563.73` | XGBoost lebih akurat ~`$110` |
| **RMSE** | **`$4,500.18`** | `$4,608.39` | XGBoost lebih stabil terhadap *outlier* |

**Kesimpulan Teknis:**
**XGBoost** terbukti lebih superior dibandingkan LightGBM di semua metrik pengujian. Model ini mampu menjelaskan **85.84%** variabilitas dari biaya klaim kesehatan, yang merupakan angka yang sangat solid untuk data asuransi yang memiliki varians alami yang tinggi.

### 2. Interpretasi Metrik dalam Konteks Asuransi

#### a. R2 Score (0.8584) - "Explanatory Power"
Model kita berhasil menangkap pola utama risiko. Artinya, fitur-fitur yang kita rekayasa (seperti `smoker_obese_interaction`, `bmi_encoded`, dll) sangat efektif. Sisa ~14% variabilitas yang tidak tertangkap kemungkinan adalah faktor acak (kecelakaan, penyakit mendadak tak terduga) atau variabel yang tidak tersedia di dataset (riwayat medis keluarga, genetik).

#### b. MAE (Mean Absolute Error): ~`$2,454`
* **Artinya:** Secara rata-rata, prediksi premi/klaim kita meleset (kurang atau lebih) sebesar **2,454 USD** dari tagihan aslinya.
* **Implikasi Bisnis:** Dalam penetapan harga (*pricing*), ini adalah rata-rata "ketidakpastian" per polis. Jika margin keuntungan asuransi per orang lebih kecil dari angka ini, perusahaan berisiko rugi pada level individu, namun bisa tertutupi oleh hukum bilangan besar (*law of large numbers*) jika agregatnya akurat.

#### c. Gap antara MAE (`$2,454`) dan RMSE (`$4,500`)
* RMSE jauh lebih tinggi daripada MAE (hampir 2x lipat).
* **Artinya:** Data klaim asuransi memiliki **Outlier Ekstrem** (Klaim Katastropik). Model terkadang melakukan kesalahan prediksi yang *sangat besar* pada kasus-kasus mahal (misalnya: memprediksi **`$10k`** padahal klaim aslinya **`$50k`**).
* **Risiko:** Model mungkin sedikit *underfitting* pada kasus penyakit berat yang biayanya meledak, yang mana wajar karena kasus tersebut jarang terjadi (*sparse data*).

### 3. Stabilitas Model (Cross-Validation Analysis)
Melihat hasil per-*fold* pada XGBoost:
* **Best Case (Fold 1):** R2 0.90 (Sangat Akurat)
* **Worst Case (Fold 2):** R2 0.82 (Cukup Akurat)
* **Analisis:** Variasi antara 0.82 hingga 0.90 menunjukkan bahwa model cukup stabil, namun ada sebagian kecil data (Fold 2) yang "sulit" diprediksi. Ini mungkin berisi kelompok pasien dengan profil aneh (misal: muda, tidak merokok, tapi klaimnya tinggi karena penyakit bawaan).

### ✅ Rekomendasi Akhir
Kita akan menggunakan **XGBoost** sebagai model final (*champion model*). Performanya secara konsisten lebih baik dalam meminimalkan risiko kesalahan harga (*pricing error*) dibandingkan LightGBM.

In [ ]:
# Visualisasi XGBoost
df_imp_xgb = plot_feature_importance(xgb_pipeline, X, y, model_name="XGBoost")

# Visualisasi LightGBM
df_imp_lgbm = plot_feature_importance(lgbm_pipeline, X, y, model_name="LightGBM")

## Feature Importance Analysis

Setelah melatih kedua model, kita meninjau fitur mana yang paling berkontribusi terhadap prediksi biaya asuransi. Visualisasi di atas menunjukkan perbedaan strategi yang mencolok antara XGBoost dan LightGBM.

### 1. XGBoost: The "Interaction Hunter"
XGBoost (Model Pemenang kita) sangat menyukai fitur-fitur hasil rekayasa (*Feature Engineering*) yang kita buat.

* **Dominasi Mutlak `smoker_obese_interaction`**:
    Fitur ini menjadi juara tak terbantahkan dengan skor *importance* melebihi **0.5**. Ini membuktikan bahwa hipotesis kita benar: Risiko (dan biaya) tertinggi bukan hanya karena merokok atau obesitas secara terpisah, melainkan **kombinasi keduanya**. XGBoost langsung mengenali bahwa jika seseorang perokok *dan* obesitas, biayanya akan meledak.
* **`bmi_smoker_continuous`**:
    Fitur terpenting kedua. Ini menegaskan bahwa bagi perokok, setiap kenaikan angka BMI berdampak sangat besar pada biaya.
* **Kesimpulan XGBoost**: Model ini bekerja sangat efektif karena kita membantunya dengan fitur interaksi. Ia "malas" melihat fitur dasar (`bmi` atau `children` sendirian) karena fitur turunan kita sudah memberikan sinyal yang jauh lebih kuat.

### 2. LightGBM: The "Raw Data" Analyst
LightGBM memiliki pendekatan yang sangat berbeda. Ia kurang tertarik pada fitur interaksi biner kita dan lebih memilih variabel kontinu asli.

* **Fokus pada `bmi` dan `age`**:
    LightGBM menempatkan `bmi` (murni) dan `age_bmi_interaction` sebagai fitur teratas. Ia mencoba memecah pohon keputusan berdasarkan angka BMI secara mendetail, bukan berdasarkan kategori "Obesitas" yang kita buat.
* **Mengabaikan Flag Perokok**:
    Yang mengejutkan, fitur terkait status perokok (`smoker_yes`, `smoker_obese_interaction`) justru dianggap tidak terlalu penting dibandingkan variabel demografis kontinu.
* **Kelemahan**: Strategi ini mungkin menjadi alasan mengapa LightGBM kalah akurat dari XGBoost. Dalam asuransi kesehatan, status perokok adalah faktor pembeda tarif yang tegas (diskrit), bukan gradasi halus. Dengan terlalu fokus pada BMI kontinu, LightGBM mungkin kehilangan ketegasan pola "High Risk Group" yang ditangkap sempurna oleh XGBoost.

### Insight & Conclusion

1.  **Validasi Feature Engineering**: Kemenangan XGBoost adalah bukti bahwa proses *feature engineering* kita (terutama pembuatan `smoker_obese_interaction`) **sangat sukses**. Tanpa fitur ini, model mungkin akan kesulitan menangkap lonjakan biaya pada kelompok risiko tinggi.
2.  **Faktor Risiko Utama**: Bagi perusahaan asuransi, data ini berteriak satu hal: **Saring ketat nasabah yang Merokok dan Obesitas.** Mereka adalah penyumbang varians klaim terbesar.
3.  **Rekomendasi**: Pertahankan fitur interaksi tersebut saat *deployment* model, karena itulah "otak" utama dari prediksi akurat XGBoost.

## Deep Learning

In [ ]:
# Run DNN Workflow
dnn_results = run_dnn_cv(
    X=X, 
    y=y, 
    num_cols=NUM_FEATS, 
    cat_cols=CAT_FEATS, 
    n_splits=5, 
    epochs=100, 
    batch_size=32
)

### Deep Neural Network (DNN) Evaluation

Kami mencoba pendekatan *Deep Learning* menggunakan arsitektur Neural Network sederhana. Berikut adalah hasil evaluasinya:

### 1. Ringkasan Performa
* **R2 Score**: **0.8446** (Lebih rendah dari XGBoost 0.8584)
* **MAE**: **`$2,659.17`** (Error lebih tinggi ~`$200` dibanding XGBoost)
* **RMSE**: **`$4,702.46`**

### 2. Mengapa DNN Tidak Optimal di Sini?

Meskipun skor R2 sebesar 0.84 menunjukkan model ini "layak", namun performanya **kalah konsisten** dibandingkan XGBoost. Ada beberapa alasan teknis mengapa DNN kesulitan mengalahkan *Gradient Boosting* pada kasus ini:

#### a. Kutukan Data Tabular (*Tabular Data Struggle*)
Deep Learning sangat superior untuk data tidak terstruktur (gambar, teks, suara). Namun, untuk **data tabular** (kolom & baris) dengan jumlah baris yang terbatas (dataset kita < 2000 baris), algoritma berbasis pohon (*Tree-based*) seperti XGBoost hampir selalu menang.
* **XGBoost** bekerja dengan membuat "pemisah" tegas (misal: Jika `smoker=yes` DAN `bmi > 30`, maka biaya tinggi). Ini sangat cocok dengan pola asuransi.
* **DNN** mencoba mencari fungsi matematika yang mulus (*smooth function*) melalui perkalian matriks (bobot & bias). DNN kesulitan menangkap "lompatan" risiko yang tajam dan diskrit pada aturan bisnis asuransi.

#### b. Instabilitas Antar Fold
Perhatikan perbedaan drastis antara **Fold 1 (R2 0.89)** dan **Fold 2 (R2 0.80)**.
Jarak performa yang jauh ini menunjukkan bahwa DNN agak **tidak stabil** dan sensitif terhadap pembagian data. Ia mungkin mengalami *overfitting* pada fold tertentu dan gagal menggeneralisasi pola pada fold lainnya.

### 3. Kesimpulan
Penggunaan Deep Learning pada kasus ini tergolong **"Overkill" namun "Underperforming"**. Kompleksitas model bertambah, waktu training lebih lama, namun akurasinya justru turun.

In [ ]:
# === Visualisasi Feature Importance DNN ===
final_dnn_model, X_dnn_proc, dnn_feats = train_final_dnn(X, y, NUM_FEATS, CAT_FEATS)
df_imp_dnn = explain_dnn_feature_importance(final_dnn_model, X_dnn_proc, y, dnn_feats)

### DNN Feature Importance

Visualisasi *Permutation Importance* pada DNN menunjukkan pola unik yang berbeda dari model *Tree-based* sebelumnya:

1.  **Dominasi `bmi_smoker_continuous`**:
    Berbeda dengan XGBoost yang menyukai fitur "aturan kaku" (`smoker_obese_interaction` berupa 0/1), DNN justru sangat bergantung pada **`bmi_smoker_continuous`**. Ini masuk akal karena Neural Network bekerja berbasis fungsi matematika dan gradien; ia lebih mudah belajar dari **angka kontinu** yang memiliki gradasi nilai daripada kategori biner yang patah-patah.

2.  **Menangkap Pola Non-Linier (`age_sq`)**:
    Fitur **`age_sq`** (usia kuadrat) muncul sebagai fitur terpenting kedua. Ini menunjukkan bahwa DNN sedang berusaha memodelkan kurva biaya yang melengkung seiring bertambahnya usia (fungsi eksponensial), sesuatu yang tidak terlalu diprioritaskan oleh model *Tree* yang hanya melakukan pemisahan data (*splitting*).

# 🥊 6. Final Evaluation and Ensembling

In [ ]:
from sklearn.ensemble import VotingRegressor

# --- 1. Setup Ulang Pipeline ML ---
# Ambil regressor terbaik yang sudah dituning
xgb_final = xgb_pipeline.named_steps['regressor']
lgbm_final = lgbm_pipeline.named_steps['regressor']

xgb_final.set_params(n_jobs=1)
lgbm_final.set_params(n_jobs=1)

# Buat Ensemble Voting
voting_reg = VotingRegressor(estimators=[('xgb', xgb_final), ('lgbm', lgbm_final)], n_jobs=1)

# Dictionary Model ML
ml_models = {
    'XGBoost': xgb_pipeline,
    'LightGBM': lgbm_pipeline,
    'Ensemble': Pipeline([('preprocessor', my_preprocessor), ('voting', voting_reg)])
}

# --- Hitung Skor ML & Gabungkan dengan DNN ---
print("🥊 FINAL BATTLE: Machine Learning vs Deep Learning")
print("-" * 50)

leaderboard = []

# Loop ML Models
scoring = {'r2': 'r2', 'mae': 'neg_mean_absolute_error', 'rmse': 'neg_root_mean_squared_error'}

for name, pipe in ml_models.items():
    print(f"Running CV for {name}...")
    cv = cross_validate(pipe, X, y, cv=5, scoring=scoring, n_jobs=1)
    leaderboard.append({
        'Model': name,
        'R2 Score': cv['test_r2'].mean(),
        'MAE': -cv['test_mae'].mean(),
        'RMSE': -cv['test_rmse'].mean()
    })

# Masukkan Hasil DNN (Dari variabel dnn_results sebelumnya)
leaderboard.append({
    'Model': 'Deep Learning (DNN)',
    'R2 Score': dnn_results['R2 Score'],
    'MAE': dnn_results['MAE'],
    'RMSE': dnn_results['RMSE']
})

# --- Tampilkan Leaderboard Akhir ---
comparison_df = pd.DataFrame(leaderboard).set_index('Model').sort_values(by='MAE')

print("\n" + "="*60)
print("🏆 FINAL LEADERBOARD (Sorted by MAE)")
print("="*60)
print(comparison_df)
print("-" * 60)

winner = comparison_df.index[0]
print(f"🎉 AND THE WINNER IS: {winner} ehehehe!")

## FINAL BATTLE ANALYSIS: Complexity vs. Efficiency

Kita telah mencapai tahap akhir. Setelah mengadu model tunggal (*Single Models*) melawan model gabungan (*Ensemble*) dan Deep Learning, berikut adalah kesimpulannya:

### 1. The Champion: XGBoost (Solo) 🏆
* **MAE: `$2,430.26`**
* **Analisis:** Secara mengejutkan (atau mungkin tidak), **XGBoost murni** keluar sebagai pemenang. Model ini bekerja sendirian tanpa perlu digabung dengan model lain.
* **Kenapa menang?** XGBoost sudah sangat optimal dalam menangkap pola diskrit (patah-patah) pada data asuransi berkat *Feature Engineering* interaksi kita. Ia tidak membutuhkan "bantuan" dari model lain untuk memperbaiki prediksinya.

### 2. The Ensemble Dilemma (Voting Regressor)
* **MAE: `$2,459.31`** (Juara 2)
* **Strategi:** Kita menggunakan `VotingRegressor` yang mengambil rata-rata prediksi dari XGBoost dan LightGBM.
* **Interpretasi Hasil:**
    Hasil Ensemble berada tepat **di tengah-tengah** antara XGBoost (terbaik) dan LightGBM (terburuk).
    * Ibaratnya: Jika Anda mencampur Jus Mangga murni yang sangat manis (XGBoost) dengan air tawar (LightGBM), hasilnya memang masih manis, tapi tidak semanis jus murninya.
    * **Pelajaran:** Ensembling bekerja paling baik jika model-model penyusunnya memiliki performa yang **setara** atau jika mereka melakukan kesalahan pada jenis data yang **berbeda** (saling melengkapi). Di sini, LightGBM hanya menarik turun rata-rata performa XGBoost.

### 3. Deep Learning (DNN)
* **MAE: `$2,659.17`** (Posisi Terakhir)
* **Analisis:** DNN konsisten berada di posisi terakhir dengan selisih error sekitar **`$230`** lebih mahal dibanding XGBoost. Ini mengonfirmasi bahwa untuk dataset tabular berukuran kecil (<2000 baris) dengan pola aturan bisnis yang tegas, *Neural Networks* bukanlah solusi yang efisien.

### Final Verdict
Kompleksitas tidak selalu menjamin akurasi. Kita tidak perlu menggunakan *Deep Learning* yang berat atau *Ensemble* yang rumit. Model **XGBoost** tunggal dengan *Feature Engineering* yang cerdas sudah cukup untuk menjadi solusi terbaik (*State-of-the-Art*) untuk permasalahan prediksi biaya asuransi ini.

# 🏁 7. Executive Summary & Conclusion

Berdasarkan rangkaian eksperimen mulai dari *preprocessing*, *feature engineering*, hingga *model comparison* (Machine Learning vs Deep Learning), berikut adalah kesimpulan strategis dari proyek ini:

### 1. Faktor Determinan Klaim Asuransi (Key Risk Drivers)
Analisis *Feature Importance* model pemenang (XGBoost) mengungkap fakta bahwa struktur biaya asuransi tidak dipengaruhi oleh faktor tunggal, melainkan **interaksi antar faktor risiko**:

* **The "Double Whammy" Effect (Perokok + Obesitas)**
    Ini adalah faktor paling mematikan dan paling mahal. Fitur `smoker_obese_interaction` mendominasi prediksi. Nasabah yang merokok **DAN** memiliki BMI tinggi (Obesitas) memiliki profil risiko yang jauh lebih tinggi secara eksponensial dibandingkan penjumlahan risiko merokok saja ditambah risiko obesitas saja.
* **Gaya Hidup > Demografi Murni**
    Faktor gaya hidup (BMI, Status Merokok) memiliki bobot prediksi yang lebih besar dibandingkan faktor demografi statis seperti `region` (wilayah tinggal) atau `sex` (jenis kelamin).
* **Penuaan (Aging)**
    Usia (`age` dan `age_sq`) tetap menjadi faktor fundamental. Biaya naik seiring bertambahnya umur, dan kenaikannya cenderung melengkung (non-linier), terutama di usia tua.

### 2. Penilaian Kelayakan Model (Model Eligibility)

Apakah model ini **LAYAK (Eligible)** untuk digunakan dalam sistem penetapan premi asuransi nyata?

**Status: ✅ ELIGIBLE WITH GUARDRAILS (Layak dengan Pengawasan)**

* **Alasan Layak:**
    * **Akurasi Tinggi (R2 ~86%):** Model mampu menjelaskan 86% variasi biaya. Dalam dunia aktuaria di mana perilaku manusia dan penyakit sangat acak, angka ini sangat solid untuk penetapan tarif dasar (*base rate*).
    * **Diskriminasi Risiko yang Tajam:** Model sangat jago membedakan mana nasabah "Murah" dan nasabah "Mahal" berkat *feature engineering* yang kuat.

* **Catatan & Risiko (Guardrails):**
    * **Margin Error (`$2,430`):** Rata-rata prediksi meleset sekitar `$2,430`. Perusahaan asuransi harus menambahkan **Risk Loading** (biaya cadangan) sebesar nilai ini ke dalam premi agar tidak rugi.
    * **Isu Outlier (RMSE Tinggi):** Karena RMSE (`$4,468`) hampir 2x lipat MAE, model terkadang masih *under-predict* pada kasus penyakit katastropik (klaim super besar).
    * **Saran Implementasi:** Gunakan prediksi model ini sebagai **acuan dasar (benchmark)**, namun untuk kasus dengan prediksi biaya sangat tinggi, tetap perlukan tinjauan manual (*human underwriter review*).